<a href="https://colab.research.google.com/github/Jegovila/SR1/blob/main/Colab/KinovaGen3Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboticstoolbox-python
!pip install ipympl

In [3]:
# Cambiar el backend para que se muestren los plots
%matplotlib widget

# Imports
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
from roboticstoolbox import RevoluteDH, PrismaticDH, DHRobot, DHLink
from spatialmath import SE3
import numpy as np

# Para desplegar los widgets
from google.colab import output
output.enable_custom_widget_manager()

# Contruir el Robot

In [6]:
qlim_j1 = [np.deg2rad(-154.1), np.deg2rad(154.1)]
qlim_j2 = [np.deg2rad(-150.1), np.deg2rad(150.1)]
qlim_j3 = [np.deg2rad(-150.1), np.deg2rad(150.1)]
qlim_j4 = [np.deg2rad(-148.98), np.deg2rad(148.98)]
qlim_j5 = [np.deg2rad(-144.97), np.deg2rad(145)]
qlim_j6 = [np.deg2rad(-148.98), np.deg2rad(148.98)]

kinova = DHRobot(
    [
        RevoluteDH(alpha = np.pi/2, d = (.1283 + .115), qlim=qlim_j1),
        RevoluteDH(alpha = np.pi, a = .280, d = 0.03, offset = np.pi/2, qlim=qlim_j2),
        RevoluteDH(alpha = np.pi/2, d = 0.02, offset = np.pi/2, qlim=qlim_j3),
        RevoluteDH(alpha = np.pi/2, d = (.14 + .105), offset = np.pi/2, qlim=qlim_j4),
        RevoluteDH(alpha = np.pi/2, d = (.0285 + .0285), offset = np.pi, qlim=qlim_j5),
        RevoluteDH(d = (.105 + .13), offset = np.pi/2, qlim=qlim_j6)
    ], name="mibot")

**Nota:** Los límites de las articulaciones "qlim" son necesarias si se queire calcular la cinemática inversa con "ikine" y para graficar el espacio de trabajo.

Imprimir la tabla DH del robot

In [ ]:
kinova

Graficar robot

In [ ]:
q = [0, 0, 0, 0, 0, 0]
kinova.plot(q)

# Cinemática Directa

In [ ]:
q = [0, 0, 0, 0, 0, 0]
T = kinova.fkine(q)
print("T=")
print(T)

## Espacio de trabajo

Las siguientes líneas grafican las posiciones que alcanza el robot. Cada ciclo es para una articulación.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in np.arange(qlim_j1[0], qlim_j1[1], 1):
  for j in np.arange(qlim_j2[0], qlim_j2[1], 1):
    for k in np.arange(qlim_j3[0], qlim_j3[1], 1):
      for l in np.arange(qlim_j4[0], qlim_j4[1], 1):
        for m in np.arange(qlim_j5[0], qlim_j5[1], 1):
          #for n in np.arange(qlim_j6[0], qlim_j6[1], 1):
          T = kinova.fkine([i,j,k,l,m,0])
          ax.scatter(T.t[0], T.t[1], T.t[2], s=1, c='red')
plt.show()

# Cinemática inversa

La orientación y la posición deseada deben ser objetos SE3

In [ ]:
Td = SE3.Trans(0.4, 0, 0.4) * SE3.Ry(np.pi/2) * SE3.Rx(-np.pi/2)
print("T deseada \n")
print(Td)

sol = kinova.ikine_LM(Td, joint_limits=True)
print(sol)

T = kinova.fkine(sol.q)
print("\nT final \n")
print(T)

Para animar la trayectoria desde una configuración inicial $q_i$ a la solución calculada. La variable "movie" nos guardará un gif que muestra mejor la animación.

In [ ]:
qi = [0, 0, 0, 0, 0, 0]
qt = rtb.jtraj(qi, sol.q, 50)
kinova.plot(qt.q, backend='pyplot', movie="mibot.gif")

# Cinemática diferencial

Para calcular el Jacobiano con respecto a la base

In [ ]:
q = [0, 0, 0, 0, 0, 0]
J = kinova.jacob0(q)
print(J)

Para el cálculo de las velocidades se requiere el vector de coordenadas generalizadas $q$ y las velocidades articulares $\dot q$

In [ ]:
q = [0, 0, 0, 0, 0, 0]
q_dot = [0.1, 0, 0.3, 0, 0, 0]

J = kinova.jacob0(q)
v = J @ q_dot
print(v)

## Cinemática Diferencial Inversa

Cálculo de la cinemática inversa utilizando la cinemática diferencial inversa.

Primero es necesario definir una función que convierta una Transformación SE3 a un vector de movimiento. La transformación va de $T_0$ a $T_1$

In [49]:
def tr2delta(T0, T1):
    delta = np.array([T1[:3, 3]-T0[:3, 3], 0.5*(np.cross(T0[:3, 0], T1[:3, 0])
                                                + np.cross(T0[:3, 1], T1[:3, 1]) + np.cross(T0[:3, 2], T1[:3, 2]))])
    return delta

$T_0$ es la base, $T_d$ es la transformación final deseada. Se calcula el vector delta deseado $x_d$

In [ ]:
T0 = SE3.Rx(0)
Td = SE3.Trans(0.4, 0, 0.4) * SE3.Ry(np.pi/2) * SE3.Rx(-np.pi/2)

T0 = np.array(T0)
Td = np.array(Td)

xd = tr2delta(T0, Td)
xd = xd.reshape((6))
print(xd)

Se calcula ahora el vector delta actual $x$

In [ ]:
qi = [0, 0, 0, 0, 0, 0]

T0 = SE3.Rx(0)
T = kinova.fkine(qi)

T0 = np.array(T0)
T = np.array(T)

x = tr2delta(T0, T)
x = x.reshape((6))
print(x)

Cinemática Diferencial Inversa

In [ ]:
iter = 1000
q = qi

for i in range(iter):

  # Cálculo de x actual
  T = np.array(kinova.fkine(q))
  x = tr2delta(T0, T).reshape((6))

  # Error
  e = xd - x

  J = kinova.jacob0(q)

  # Pseudoinversa del Jacobiano (O transpuesta)
  Ji = np.linalg.pinv(J)
  #Ji = np.transpose(J)
  #Ji = np.linalg.inv((np.transpose(J) @ J) + 4 * np.eye(np.max([J.shape[0], J.shape[1]]))) @ np.transpose(J)

  ## SOLO RESOLVER POSICIÓN
  # qp = .2 * (Ji[:,:3] @ e[:3])
  # q[:3] = q[:3] + qp[:3]

  ## RESOLVER POSICIÓN Y ORIENTACIÓN
  qp = .2 * (Ji @ e)
  q = q + qp

solution = q
print(solution)
print(sol.q)

Comparamos $T$ con $T_d$

In [ ]:
print("T\n")
print(kinova.fkine(solution))

print("\nT_d\n")
print(Td)

Por último, trazar su trayectoria

In [ ]:
qt = rtb.jtraj(qi, solution, 50)
kinova.plot(qt.q, backend='pyplot', movie="mibot.gif")